In [1]:
#Cargar el data Set mediante el comprimido en google drive
#Link público https://drive.google.com/file/d/16ajJdjHzPr_WwfauEOv5dKZ1IKYoMQyk/view?usp=sharing
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
#Descomprimimos el dataSet
import os
import zipfile


ubicacion_zip = '/content/gdrive/My Drive/chest_xray.zip'
ubicacion_folder_unzip = '/content/chest_xray'

zip_ref = zipfile.ZipFile(ubicacion_zip, 'r')
zip_ref.extractall(ubicacion_folder_unzip)
zip_ref.close()

In [0]:
#asignación de las carpetas de entrenamiento
train_normal_dir = os.path.join(ubicacion_folder_unzip+'/chest_xray/train/NORMAL')
train_neumonia_dir = os.path.join(ubicacion_folder_unzip+'/chest_xray/train/PNEUMONIA')

test_normal_dir = os.path.join(ubicacion_folder_unzip+'/chest_xray/test/NORMAL')
test_neumonia_dir = os.path.join(ubicacion_folder_unzip+'/chest_xray/test/PNEUMONIA')

val_normal_dir = os.path.join(ubicacion_folder_unzip+'/chest_xray/val/NORMAL')
val_neumonia_dir = os.path.join(ubicacion_folder_unzip+'/chest_xray/val/PNEUMONIA')

In [0]:
#Entrenamiento de las imagenes normales
train_normal_names = os.listdir(train_normal_dir)
print(train_normal_names[:10])

test_normal_names = os.listdir(test_normal_dir)
print(test_normal_names[:10])

val_normal_names = os.listdir(val_normal_dir)
print(val_normal_names[:10])

In [0]:
#Entrenamiento de las imagenes pneumonía
train_neumonia_names = os.listdir(train_neumonia_dir)
print(train_neumonia_names[:10])

test_neumonia_names = os.listdir(test_neumonia_dir)
print(test_neumonia_names[:10])

val_neumonia_names = os.listdir(val_neumonia_dir)
print(val_neumonia_names[:10])

In [0]:
#Importar numpy para el calculo dce matrices, tf para las redes neuronales, keras para el procesamiento de imagnes
#y matplotlib para el procesamiento de imagenes
import os
import numpy as np

import tensorflow as tf

import keras
from keras.optimizers import RMSprop
from keras.preprocessing.image import ImageDataGenerator

import matplotlib.pyplot as plt
import matplotlib.image as mpimg

In [0]:
#Número de columnas a mostar en el dataSet ejemplo de imagnes 4x4
nrows = 4
ncols = 4

pic_index = 0

In [0]:
#Recorre el directorio train e imprime la muestra ejemplo 
fig = plt.gcf()
fig.set_size_inches(ncols * 2, nrows * 2)

pic_index += 8
next_normal_pix = [os.path.join(train_normal_dir, fname) for fname in train_normal_names[pic_index-8:pic_index]]
next_neumonia_pix = [os.path.join(train_neumonia_dir, fname) for fname in train_neumonia_names[pic_index-8:pic_index]]

fig.suptitle("Dataset TRAIN")

for i, img_path in enumerate(next_normal_pix+next_neumonia_pix):
    sp = plt.subplot(nrows, ncols, i + 1)
    sp.axis('Off')
    plt.title(img_path.split("/")[2])
    img = mpimg.imread(img_path)
    plt.imshow(img, cmap='gray')

plt.show()

In [0]:
#Recorre el directorio test e imprime la muestra ejemplo 
fig = plt.gcf()
fig.set_size_inches(ncols * 2, nrows * 2)

pic_index += 8
next_normal_pix = [os.path.join(test_normal_dir, fname) for fname in test_normal_names[pic_index-8:pic_index]]
next_neumonia_pix = [os.path.join(test_neumonia_dir, fname) for fname in test_neumonia_names[pic_index-8:pic_index]]

fig.suptitle("Dataset TEST")

for i, img_path in enumerate(next_normal_pix+next_neumonia_pix):
    sp = plt.subplot(nrows, ncols, i + 1)
    sp.axis('Off')
    plt.title(img_path.split("/")[2])
    img = mpimg.imread(img_path)
    plt.imshow(img, cmap='gray')

plt.show()

In [0]:
#Recorre el directorio val e imprime la muestra ejemplo 
fig = plt.gcf()
fig.set_size_inches(ncols * 2, nrows * 2)

pic_index += 8
next_normal_pix = [os.path.join(val_normal_dir, fname) for fname in val_normal_names[pic_index-8:pic_index]]
next_neumonia_pix = [os.path.join(val_neumonia_dir, fname) for fname in val_neumonia_names[pic_index-8:pic_index]]

fig.suptitle("Dataset VAL")

for i, img_path in enumerate(next_normal_pix+next_neumonia_pix):
    sp = plt.subplot(nrows, ncols, i + 1)
    sp.axis('Off')
    plt.title(img_path.split("/")[2])
    img = mpimg.imread(img_path)
    plt.imshow(img, cmap='gray')

plt.show()

In [0]:
# Todas las imágenes serán reescaladas por 1./255
train_datagen = ImageDataGenerator(rescale=1/255)
test_datagen = ImageDataGenerator(rescale=1/255)
val_datagen = ImageDataGenerator(rescale=1/255)

# Carpeta donde se encuentran las imagenes de entrenamiento
train_dir = os.path.join(ubicacion_folder_unzip+ '/chest_xray/train/')
test_dir = os.path.join(ubicacion_folder_unzip+ '/chest_xray/test/')
val_dir = os.path.join(ubicacion_folder_unzip+ '/chest_xray/val/')

train_generator = train_datagen.flow_from_directory(
        train_dir,  
        target_size=(28, 28),  # alterando este parametro podemos redimensionar las imagenes desde la lectura
        batch_size=128, # dejar fijo
        class_mode='binary') #clasificacion binaria

test_generator = test_datagen.flow_from_directory(
        test_dir,  
        target_size=(28, 28),  # alterando este parametro podemos redimensionar las imagenes desde la lectura
        batch_size=128, # dejar fijo
        class_mode='binary') #clasificacion binaria

val_generator = val_datagen.flow_from_directory(
        val_dir,  
        target_size=(28, 28),  # alterando este parametro podemos redimensionar las imagenes desde la lectura
        batch_size=128, # dejar fijo
        class_mode='binary') #clasificacion binaria

In [0]:
#Creación de la red neuronal convolutiva y un activador sigmoid
model = keras.Sequential()

model.add(keras.layers.Conv2D(64, (3,3), activation='relu', input_shape=(28, 28, 3)))
model.add(keras.layers.MaxPooling2D(2, 2))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(128, activation=tf.nn.relu)) #Con 128 neuronas
model.add(keras.layers.Dense(128, activation=tf.nn.relu)) #Agregamos una capa adicional con 128 neuronas
model.add(keras.layers.Dense(1, activation=tf.nn.sigmoid))

In [0]:
#Compilación del modelo con el optimizador Adam
model.compile(loss='binary_crossentropy',
              #optimizer=RMSprop(lr=0.001),
              optimizer = keras.optimizers.Adam(lr=0.001),
              metrics=['acc'])

In [0]:
#Entrenamiento del modelo con train
history = model.fit_generator(train_generator, steps_per_epoch=20, epochs=20, verbose=1)

In [0]:
#Convergencia del entrenamiento para train
plt.plot(history.history['loss'])
plt.show()

In [0]:
#Entrenamiento del modelo con test
history = model.fit_generator(test_generator, steps_per_epoch=20, epochs=20, verbose=1)

In [0]:
#Convergencia del entrenamiento para test
plt.plot(history.history['loss'])
plt.show()

In [0]:
#Entrenamiento del modelo con val
history = model.fit_generator(val_generator, steps_per_epoch=20, epochs=20, verbose=1)

In [0]:
#Convergencia del entrenamiento para val
plt.plot(history.history['loss'])
plt.show()